In [1]:
from basf_measurement_parser import BASFMeasurementParser
from prompt_evaluation import *
from prompt_builder import *
from logger import ParserLogger



/Users/admin/opt/anaconda3/envs/langchain/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [8]:
# logger = ParserLogger("Data/text_data_ipa221229_30_patents.log")

chat_prompt, prompt, examples  = PromptBuilder.build_prompt_from_dir("002")

In [9]:
prompt

{'text': 'You are given a text from chemistry research patent, extract all the measurements mentioned in the text. Only Extract Measurement, Unit and Value ranges.\nOnly extract measurements that have value and units.\nIf you find no measurments, return [].',
 'output_format': "Your output should follow be a list of json objects, good output format : [ { 'measurement ' : 'string ', 'unit ' : 'string ', 'value ' : 'string ' }, { 'measurement ' : 'string ', 'unit ' : 'string ', 'value ' : 'string ' }",
 'id': '000'}

In [10]:
measurement_parser = BASFMeasurementParser(prompt=chat_prompt, prompt_id="002", chunk_size=6000)


In [5]:
text = measurement_parser._load_txt_and_split("Data/text_data_ipa221229_5_patents.txt")

In [11]:
patents_text_results = measurement_parser.parse_txt_by_chunks(file_path="Data/text_data_ipa221229_5_patents.txt", output_format=prompt["output_format"])


AttributeError: 'list' object has no attribute 'replace'

In [4]:
file = open("output/parsing_results_54_chunks_prompt_001.txt")
prediction_test = file.read()

In [5]:
prediction_test_list = prediction_test.replace("\n", " ").split(']')
prediction_test_list[0]

'[     {         "measurement": "surface area",          "unit": "m^2/g",          "value": "between about 5 and about 350"     },      {         "measurement": "surface area",          "unit": "m/g",          "value": "between about 30 and 80"     },     {         "measurement": "coated Substrate support",          "unit": "g/in",          "value": "about 1.5 to about 7.0"     },     {         "measurement": "polypeptide concentration",          "unit": "g/ml",          "value": "1 to 9"     } '

In [6]:
import re
import ast
import string 

def build_dict_from_json_string(json_str_list):
    try:
        fixed_json = json.loads(json_str_list)
        return fixed_json
    except json.JSONDecodeError as e:
        # Remove the surrounding square brackets and newlines
        clean_string = json_str_list.replace("'", "\"")

        # Safely evaluate the string as a list of dictionaries
        dictionary_list = ast.literal_eval(clean_string)
        
        return dictionary_list

In [7]:
prediction = '''[{"measurement": "nucleic acid sequencing", "unit": "", "value": ""},
{"measurement": "rt-pcr sequencing", "unit": "", "value": ""},
{"measurement": "pacbio sequencing protocol", "unit": "", "value": ""},
{"measurement": "geographic location of infected individuals", "unit": "", "value": ""},
{"measurement": "barcodes linked to zip code or geographic identifier", "unit": "", "value": ""},
{"measurement": "prevalence of variants in infected individuals or general population", "unit": "", "value": ""},
{"measurement": "correlation of specific variants with infectivity and disease severity", "unit": "", "value": ""},
{"measurement": "data deposition in CDC database", "unit": "", "value": ""}]'''

In [8]:
prediction_test_list[30]

' As there are no measurements with units and values in the provided text, no output can be generated. "measurement": "top lance diameter", "unit": "NA", "value": "larger than sublance diameter", "measurement": "sublance diameter", "unit": "NA", "value": "NA", "measurement": "probe parameters", "unit": "NA", "value": "NA", "measurement": "sampling tool parameters", "unit": "NA", "value": "NA" I apologize, but I cannot provide an answer to your request as it seems incomplete and does not provide any text to extract measurements from. Could you please provide the appropriate text? [     {         "measurement": "diameter of top lance distal end",         "unit": null,         "value": "d1"     },     {         "measurement": "maximum diameter of top lance coupling portion",         "unit": null,         "value": "d1"     },     {         "measurement": "diameter of sublance cavity",         "unit": null,         "value": "d2"     },     {         "measurement": "diameter of external cros

In [9]:
import json
def post_process_predicions(predictions):
    
    predictions_json = []
    false_jsons =[]
    for i, prediction in enumerate(prediction_test_list):
        try:
            predictions_json.extend(build_dict_from_json_string(prediction+']'))
        except Exception as e:
#         print(e)
            false_jsons.append(prediction)
    return predictions_json, false_jsons

In [10]:
predictions_json, false_jsons = post_process_predicions(prediction_test_list)

In [11]:
import pandas as pd 

pd.DataFrame(predictions_json)

,measurement,unit,value,measurement,unit,value
0,surface area,m^2/g,between about 5 and about 350,NaN,NaN,NaN
1,surface area,m/g,between about 30 and 80,NaN,NaN,NaN
2,coated Substrate support,g/in,about 1.5 to about 7.0,NaN,NaN,NaN
3,polypeptide concentration,g/ml,1 to 9,NaN,NaN,NaN
4,nucleic acid sequencing,,,NaN,NaN,NaN
...,...,...,...,...,...,...
498,yield strength,mpa,230 or more,NaN,NaN,NaN
499,tensile strength,mpa,380 or more,NaN,NaN,NaN
500,heating temperature,c,1.0 to 1.20,NaN,NaN,NaN
501,hot rolling temperature,c,80 or more,NaN,NaN,NaN


In [12]:
predictions_json = []
false_jsons =[]
for i, prediction in enumerate(prediction_test_list):
    try:
        predictions_json.extend(build_dict_from_json_string(prediction+']'))
    except Exception as e:
        print(e)
        false_jsons.append(prediction)

invalid syntax (<unknown>, line 1)
invalid syntax (<unknown>, line 1)
invalid syntax (<unknown>, line 1)
invalid syntax (<unknown>, line 1)
'{' was never closed (<unknown>, line 1)
unmatched ']' (<unknown>, line 1)


In [13]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

# The schema I want out
response_schemas = [
    ResponseSchema(name="measurements", description="Measurements details"),
    ResponseSchema(name="unit", description="Unit used in the measurments"),
    ResponseSchema(name="value", description="The value of the measurements")
]

# The parser that will look for the LLM output in my schema and return it back to me
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [23]:
from dotenv import load_dotenv
load_dotenv("env.env")


True

In [20]:
# Build vectorstore and keep the metadata
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=all_splits,
                                    embedding=OpenAIEmbeddings(deployment_name=""))

NameError: name 'all_splits' is not defined